# Homework 04. 

#### MSCA 32013 Optimization and Simulation Methods. University of Chicago.

#### Group 7. Ben Ossyra, Peter Pezon, Yawen Zhang

## Q1. Sudoku

Solve Sudoku using integer programming. Example is on Julia/JuMP, but don't look at it.
Each row has to have columns 1 - 9 - in no particular order.
Each 3x3 grid has to have numbesr 1 - 9, in no particular order.
Fill in zero with the numbers.

Encoding. Vector of 1x9. Takes value of 0 and 1. 

Represent 1 as [1 0 0 0 0 ...]

Represent 2 as [0 1 0 0 0 ...]

Need to put a number in the sudoku grid. Represent number as vector.

Also need to put number in a position.

Position: Matrix 9x9 

Put number in row, cell : 0 or 1. binary.

$ x[i,j,s] = 1 $

Example:

$ x[7,4,6] = 1 $ represents number 6 in row 7 column 4. 

Row, column and value

That's the decision: whether to put 6 on a paper ... 

729 variables: 9 x 9 x 9

VARIABLES ARE DIFFERENT FROM CONSTRAINTS.

Constraints are initial values

Constriants:
- Rows sum to 9
- Cols sum to 9
- Grids sum to 9
- Integer vectors
- Initial values


How to make sure digit is used only once?

$ x[1,1,1] + x[1,2,1] + ... + x[1,9,1] = 1 $
$ x[1,1,2] + x[1,2,2] + ... + x[1,9,2] = 1 $

This works bc value can either be 1 or 0, and if digit can only appear once, then the sum is 1.

Repeat for columns:

$ x[1,1,1] + x[2,1,1] + ... + x[9,1,1] = 1 $
$ x[1,1,2] + x[2,1,2] + ... + x[9,1,2] = 1 $

Guarantees that #1 appears only once in the column; that #2 appears only once in the column; etc...

Theres only one unique solution so doesnt matter if we maximize or minimize... 

In [51]:
using JuMP, GLPK, Test

m = Model(GLPK.Optimizer)

I = [
        3	1	0	0	5	8	0	0	4;
        0	0	9	3	2	0	0	0	0;
        0	2	5	1	0	4	0	9	0;
        0	0	0	0	0	0	3	8	9;
        0	0	8	0	0	0	5	0	0;
        5	4	6	0	0	0	0	0	0;
        0	8	0	2	0	3	6	5	0;
        0	0	0	0	7	1	4	0	0;
        7	0	0	4	8	0	0	2	1 
]

# 3x3 matrix indexed by i = row, j = col, d = digit
@variable(m, x[1:9, 1:9, 1:9], Bin)

# each cell must contain only one digit
for i=1:9
    for j=1:9
        @constraint(m, sum(x[i,j,:]) == 1)
    end
end

# each column contains the digits 1 to 9
for i=1:9
    for k=1:9
        @constraint(m, sum(x[i,:,k]) == 1)
    end
end

# each row contains the digits 1 to 9
for j=1:9
    for k=1:9
        @constraint(m, sum(x[:,j,k]) == 1)
    end
end

# each subgrid contains the digits 1 to 9
for i=1:3:7 # subsetting row.. skipping: 1, 4, 7
    for j=1:3:7 # subsetting col.. skipping: 1, 4, 7
        for k=1:9
            @constraint(m, sum(x[i:i + 2, j:j + 2, k]) == 1)
        end
    end
end

# cannot change the cells initially fixed
for i=1:9
    for j=1:9
        if I[i,j] > 0
            # must match initial vector
            @constraint(m, x[i,j,I[i,j]] == 1)
        end
    end
end

# Set the objective:
@objective(m, Min, sum(x[i,j,k] for i=1:9 for j=1:9 for k=1:9))

# Print the model:
print(m)

# Solve the optimization problem:
JuMP.optimize!(m)

# Print the objective value:
println("Objective value: ", JuMP.objective_value(m))

# Print the optimal solutions obtained
println("x = ", JuMP.value.(x))

term_status = JuMP.termination_status(m)
primary_status = JuMP.primal_status(m)
is_optimal = term_status == MOI.OPTIMAL
print(term_status, primary_status, is_optimal)

Objective value: 81.0
x = [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0;;; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0

The optimal solution: convert vectors to digits.

In [60]:
soln = JuMP.value.(x)
soln_digits = zeros(9,9)

for i=1:9
    for j=1:9
        soln_digits[i,j] = argmax(soln[i,j,:])
    end
end
optSol

9×9 Matrix{Float64}:
 3.0  1.0  7.0  9.0  5.0  8.0  2.0  6.0  4.0
 4.0  6.0  9.0  3.0  2.0  7.0  8.0  1.0  5.0
 8.0  2.0  5.0  1.0  6.0  4.0  7.0  9.0  3.0
 2.0  7.0  1.0  6.0  4.0  5.0  3.0  8.0  9.0
 9.0  3.0  8.0  7.0  1.0  2.0  5.0  4.0  6.0
 5.0  4.0  6.0  8.0  3.0  9.0  1.0  7.0  2.0
 1.0  8.0  4.0  2.0  9.0  3.0  6.0  5.0  7.0
 6.0  9.0  2.0  5.0  7.0  1.0  4.0  3.0  8.0
 7.0  5.0  3.0  4.0  8.0  6.0  9.0  2.0  1.0